In [1]:
import numpy as np
import xlrd

In [2]:
doc = xlrd.open_workbook('指挥喵/天赋.xlsx')

In [3]:
sheet = doc.sheet_by_index(0)
print(sheet.name, sheet.ncols, sheet.nrows)

Sheet1 18 56


In [5]:
def parsed(value):
    if isinstance(value, str):
        return value.strip()
    return value
    
def get_cell(i, j):
    for x0, x1, y0, y1 in sheet.merged_cells:
        if x0 == i and y0 == j:
            break
        if x0 <= i < x1 and y0 <= j < y1:
            return parsed(sheet.cell(x0, y0).value)
    return parsed(sheet.cell(i, j).value)


In [6]:
data = []
for i in range(sheet.nrows):
    row = []
    for j in range(sheet.ncols):
        row.append(get_cell(i, j))
    data.append(row)

In [7]:
names = data[2]
split_idx = [4, 16]

a, b = split_idx
desc = names[:a]
score = names[a:b]
chance = names[b:]


In [8]:
info = {}
for line in data[3:]:
    if not any(line):
        continue
    item = {
        'name': line[0],
        'desc': dict(zip(names, line[:a])),
        'scores': dict(zip(score, line[a:b])),
        'chance': dict(zip(chance, line[b:])),
    }
    name = item['name']
    if name not in info:
        info[name] = item
    else:
        info[name]['desc'].append(item['desc'])
        for key in score:
            if item['scores'][key]:
                info[name]['scores'][key] = item['scores'][key]
    if name == '':
        print(line)
        
for item in info.values():
    for key in score:
        if item['scores'][key] == '':
            item['scores'][key] = 0


In [9]:
info

{'被期待的新星': {'name': '被期待的新星',
  'desc': {'天赋名': '被期待的新星', '简称': '-', '描述': '自身获得的经验值提高10%', '特殊天赋': 1.0},
  'scores': {'潜艇': 0,
   '驱逐': 0,
   '轻雷巡': 0,
   '重雷巡': 0,
   '炮巡': 0,
   '轻航': 0,
   '正航': 0,
   '战列': 0,
   '战巡': 0,
   '航战': 0,
   '重炮': 0,
   '维修': 0},
  'chance': {'SR': 1.0, 'SSR': 1.0}},
 '最佳玩伴': {'name': '最佳玩伴',
  'desc': {'天赋名': '最佳玩伴', '简称': '-', '描述': '作为陪玩单位时提供经验提高10%', '特殊天赋': 1.0},
  'scores': {'潜艇': 0,
   '驱逐': 0,
   '轻雷巡': 0,
   '重雷巡': 0,
   '炮巡': 0,
   '轻航': 0,
   '正航': 0,
   '战列': 0,
   '战巡': 0,
   '航战': 0,
   '重炮': 0,
   '维修': 0},
  'chance': {'SR': 1.0, 'SSR': 0.0}},
 '其疾如风': {'name': '其疾如风',
  'desc': {'天赋名': '其疾如风', '简称': '风', '描述': '舰队航速提高3点', '特殊天赋': 1.0},
  'scores': {'潜艇': 1.0,
   '驱逐': 1.0,
   '轻雷巡': 2.0,
   '重雷巡': 3.0,
   '炮巡': 2.0,
   '轻航': 3.0,
   '正航': 3.0,
   '战列': 3.0,
   '战巡': 3.0,
   '航战': 3.0,
   '重炮': 3.0,
   '维修': 3.0},
  'chance': {'SR': 1.0, 'SSR': 1.0}},
 '其徐如林': {'name': '其徐如林',
  'desc': {'天赋名': '其徐如林',
   '简称': '林',
   '描述': '舰队防空、反潜提高15

In [10]:
fleet = {
    '驱逐': 3,
    '战列': 3,
}
def sort_talents(fleet, talents=info):
    def key(item):
        item['ScoreInfo'] = []
        score = 0
        for key in fleet:
            score_add = item['scores'][key] * fleet[key]
            if score_add:
                item['ScoreInfo'].append((key, fleet[key], item['scores'][key],))
                score += score_add
        item['Score'] = score
        return -score
    items = list(talents.values())
    items.sort(key=key)
    items = [{'score': item['Score'], 'name': item['name'], 'scoreInfo': item['ScoreInfo']} for item in items if item['Score'] > 0]
    return items

def show_scores(scores):
    for i, item in enumerate(scores):
        print('{1:2d}: {0[score]:2.0f}, {0[name]:}  {0[scoreInfo]}'.format(item, i+1))
    print()
        
        
def show_talents(fleet, talents=info, limit=None):
    print(fleet)
    items = sort_talents(fleet, talents)
    if limit:
        items = items[:limit]
    show_scores(items)

In [19]:
short_dict = {
    '航': {'正航', '轻航'},
    '战': {'战列', '战巡', '航战'},
    '驱': {'驱逐'},
    '巡': {'轻雷巡', '重雷巡', '炮巡'},
    '潜': {'潜艇'}
}

def search_talent(ship_types, talent_short):
    candidates = []
    for talent in info.values():
        if talent_short == talent['desc']['简称']:
            candidates.append(talent)
    if len(candidates) == 1:
        return candidates[0]['name']
    for talent in candidates:
        for ship_type in ship_types:
            if talent['scores'][ship_type] > 0:
                return talent['name']
    return {"Error": {'ship_types': ship_types, 'talent_short': talent_short}, 'candidates': candidates}
    
def get_talents(cat):
    s_type, s_talents = cat.split(' ')
    ship_types = short_dict[s_type]
    
    talents = []
    for s in s_talents:
        talents.append(search_talent(ship_types, s))
    return talents

## 可能的舰队配置

In [44]:
fleet_main = [
    {"战列": 1},
    {"正航": 1},
    {"轻航": 1},
    
    {"战列": 2},
    {"战列": 1, "正航": 1},
    {"战列": 1, "轻航": 1},
    {"战列": 1, "维修": 1},
    
    {"战列": 3},
    {"正航": 3},
    {"正航": 2, "轻航": 1},
    {"战列": 1, "正航": 2},
    {"战列": 1, "正航": 1, "轻航": 1},
    {"战列": 1, "正航": 1, "维修": 1},
    {"战列": 1, "轻航": 1, "维修": 1},
]
fleet_scout = [
    {"驱逐": 1},
    
    {"驱逐": 2},
    {"炮巡": 2},
    {"驱逐": 1, "炮巡": 1},
    
    {"驱逐": 3},
    {"炮巡": 3},
    {"驱逐": 2, "炮巡": 1},
    {"驱逐": 2, "轻雷巡": 1},
    {"炮巡": 2, "轻雷巡": 1},
    {"炮巡": 1, "轻雷巡": 2},
    {"炮巡": 2, "重雷巡": 1},
    {"炮巡": 1, "重雷巡": 1, "轻雷巡": 1},
    {"驱逐": 1, "重雷巡": 1, "轻雷巡": 1},
]

fleets = []
for main in fleet_main:
    for scout in fleet_scout:
        fleets.append(dict(**scout, **main))

## 各种配置下的最优技能

In [45]:
for fleet in fleets:
    show_talents(fleet, limit=15)

{'驱逐': 1, '战列': 1}
 1: 20, 侵略如火  [('驱逐', 1, 10.0), ('战列', 1, 10.0)]
 2: 15, 既定的命运  [('驱逐', 1, 7.0), ('战列', 1, 8.0)]
 3: 12, 其徐如林  [('驱逐', 1, 6.0), ('战列', 1, 6.0)]
 4: 12, 不动如山  [('驱逐', 1, 6.0), ('战列', 1, 6.0)]
 5: 12, 小小的奇迹  [('驱逐', 1, 7.0), ('战列', 1, 5.0)]
 6:  9, 水雷魂  [('驱逐', 1, 9.0)]
 7:  9, 一发入魂  [('战列', 1, 9.0)]
 8:  8, 炮击新手·主力  [('战列', 1, 8.0)]
 9:  8, 新晋指挥官·驱逐  [('驱逐', 1, 8.0)]
10:  7, 新人雷击士·驱逐  [('驱逐', 1, 7.0)]
11:  7, 轮机手·驱逐  [('驱逐', 1, 7.0)]
12:  7, 新晋指挥官·皇家  [('驱逐', 1, 3.0), ('战列', 1, 4.0)]
13:  7, 新晋指挥官·铁血  [('驱逐', 1, 4.0), ('战列', 1, 3.0)]
14:  6, 装填新手·战列  [('战列', 1, 6.0)]
15:  6, 新手观测士·主力  [('战列', 1, 6.0)]

{'驱逐': 2, '战列': 1}
 1: 30, 侵略如火  [('驱逐', 2, 10.0), ('战列', 1, 10.0)]
 2: 22, 既定的命运  [('驱逐', 2, 7.0), ('战列', 1, 8.0)]
 3: 19, 小小的奇迹  [('驱逐', 2, 7.0), ('战列', 1, 5.0)]
 4: 18, 其徐如林  [('驱逐', 2, 6.0), ('战列', 1, 6.0)]
 5: 18, 不动如山  [('驱逐', 2, 6.0), ('战列', 1, 6.0)]
 6: 18, 水雷魂  [('驱逐', 2, 9.0)]
 7: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 8: 14, 新人雷击士·驱逐  [('驱逐', 2, 7.0)]
 9: 14, 轮机手·驱逐

 3: 36, 既定的命运  [('驱逐', 2, 7.0), ('炮巡', 1, 8.0), ('战列', 1, 8.0), ('轻航', 1, 6.0)]
 4: 29, 其徐如林  [('驱逐', 2, 6.0), ('炮巡', 1, 5.0), ('战列', 1, 6.0), ('轻航', 1, 6.0)]
 5: 23, 小小的奇迹  [('驱逐', 2, 7.0), ('炮巡', 1, 2.0), ('战列', 1, 5.0), ('轻航', 1, 2.0)]
 6: 18, 水雷魂  [('驱逐', 2, 9.0)]
 7: 18, 新晋指挥官·白鹰  [('驱逐', 2, 3.0), ('炮巡', 1, 3.0), ('战列', 1, 3.0), ('轻航', 1, 6.0)]
 8: 17, 新晋指挥官·重樱  [('驱逐', 2, 4.0), ('炮巡', 1, 3.0), ('战列', 1, 2.0), ('轻航', 1, 4.0)]
 9: 17, 新晋指挥官·铁血  [('驱逐', 2, 4.0), ('炮巡', 1, 4.0), ('战列', 1, 3.0), ('轻航', 1, 2.0)]
10: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
11: 15, 新晋指挥官·皇家  [('驱逐', 2, 3.0), ('炮巡', 1, 4.0), ('战列', 1, 4.0), ('轻航', 1, 1.0)]
12: 14, 新人雷击士·驱逐  [('驱逐', 2, 7.0)]
13: 14, 轮机手·驱逐  [('驱逐', 2, 7.0)]
14: 13, 其疾如风  [('驱逐', 2, 1.0), ('炮巡', 1, 2.0), ('战列', 1, 3.0), ('轻航', 1, 3.0), ('维修', 1, 3.0)]
15: 10, 炮击新手·驱逐  [('驱逐', 2, 5.0)]

{'驱逐': 2, '轻雷巡': 1, '战列': 1, '轻航': 1, '维修': 1}
 1: 50, 侵略如火  [('驱逐', 2, 10.0), ('轻雷巡', 1, 10.0), ('战列', 1, 10.0), ('轻航', 1, 10.0)]
 2: 36, 不动如山  [('驱逐', 2, 6.0), ('轻雷

In [12]:
show_talents({'炮巡': 3, '战列': 2, '轻航': 1}, limit=15)

{'炮巡': 3, '战列': 2, '轻航': 1}
 1: 60, 侵略如火  [('炮巡', 3, 10.0), ('战列', 2, 10.0), ('轻航', 1, 10.0)]
 2: 46, 既定的命运  [('炮巡', 3, 8.0), ('战列', 2, 8.0), ('轻航', 1, 6.0)]
 3: 36, 不动如山  [('炮巡', 3, 6.0), ('战列', 2, 6.0), ('轻航', 1, 6.0)]
 4: 33, 其徐如林  [('炮巡', 3, 5.0), ('战列', 2, 6.0), ('轻航', 1, 6.0)]
 5: 24, 见敌必战  [('炮巡', 3, 8.0)]
 6: 21, 炮击新手·巡洋  [('炮巡', 3, 7.0)]
 7: 21, 新晋指挥官·巡洋  [('炮巡', 3, 7.0)]
 8: 21, 新晋指挥官·白鹰  [('炮巡', 3, 3.0), ('战列', 2, 3.0), ('轻航', 1, 6.0)]
 9: 21, 新晋指挥官·皇家  [('炮巡', 3, 4.0), ('战列', 2, 4.0), ('轻航', 1, 1.0)]
10: 20, 新晋指挥官·铁血  [('炮巡', 3, 4.0), ('战列', 2, 3.0), ('轻航', 1, 2.0)]
11: 18, 小小的奇迹  [('炮巡', 3, 2.0), ('战列', 2, 5.0), ('轻航', 1, 2.0)]
12: 18, 一发入魂  [('战列', 2, 9.0)]
13: 17, 新晋指挥官·重樱  [('炮巡', 3, 3.0), ('战列', 2, 2.0), ('轻航', 1, 4.0)]
14: 16, 炮击新手·主力  [('战列', 2, 8.0)]
15: 15, 其疾如风  [('炮巡', 3, 2.0), ('战列', 2, 3.0), ('轻航', 1, 3.0)]



In [13]:
show_talents({'驱逐': 3})

{'驱逐': 3}
 1: 30, 侵略如火  [('驱逐', 3, 10.0)]
 2: 27, 水雷魂  [('驱逐', 3, 9.0)]
 3: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 4: 21, 小小的奇迹  [('驱逐', 3, 7.0)]
 5: 21, 既定的命运  [('驱逐', 3, 7.0)]
 6: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 7: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 8: 18, 其徐如林  [('驱逐', 3, 6.0)]
 9: 18, 不动如山  [('驱逐', 3, 6.0)]
10: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
11: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
12: 12, 新晋指挥官·重樱  [('驱逐', 3, 4.0)]
13: 12, 新晋指挥官·铁血  [('驱逐', 3, 4.0)]
14:  9, 新晋指挥官·白鹰  [('驱逐', 3, 3.0)]
15:  9, 新晋指挥官·皇家  [('驱逐', 3, 3.0)]
16:  3, 其疾如风  [('驱逐', 3, 1.0)]
17:  3, 对空炮手·先锋  [('驱逐', 3, 1.0)]
18:  3, 声纳兵·先锋  [('驱逐', 3, 1.0)]
19:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]
20:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]



In [14]:
show_talents({'驱逐': 3, '战列': 0}, limit=15)

{'驱逐': 3, '战列': 0}
 1: 30, 侵略如火  [('驱逐', 3, 10.0)]
 2: 27, 水雷魂  [('驱逐', 3, 9.0)]
 3: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 4: 21, 小小的奇迹  [('驱逐', 3, 7.0)]
 5: 21, 既定的命运  [('驱逐', 3, 7.0)]
 6: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 7: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 8: 18, 其徐如林  [('驱逐', 3, 6.0)]
 9: 18, 不动如山  [('驱逐', 3, 6.0)]
10: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
11: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
12: 12, 新晋指挥官·重樱  [('驱逐', 3, 4.0)]
13: 12, 新晋指挥官·铁血  [('驱逐', 3, 4.0)]
14:  9, 新晋指挥官·白鹰  [('驱逐', 3, 3.0)]
15:  9, 新晋指挥官·皇家  [('驱逐', 3, 3.0)]



In [16]:
talents = ['装填新手·战列', '轮机手·战列', '见敌必战']
# talents = ['炮击新手·驱逐', '新晋指挥官·驱逐', '轮机手·驱逐']
talents = {key: info[key] for key in talents}
scores = sort_talents({'炮巡': 1, '轻雷巡': 2, '战列': 3}, talents)
sum([item['score'] for item in scores]), scores

(48.0,
 [{'score': 24.0,
   'name': '见敌必战',
   'scoreInfo': [('炮巡', 1, 8.0), ('轻雷巡', 2, 8.0)]},
  {'score': 18.0, 'name': '装填新手·战列', 'scoreInfo': [('战列', 3, 6.0)]},
  {'score': 6.0, 'name': '轮机手·战列', 'scoreInfo': [('战列', 3, 2.0)]}])

In [17]:
talents = ['既定的命运', '操舵手·小型舰']
# talents = ['炮击新手·驱逐', '新晋指挥官·驱逐', '轮机手·驱逐']
talents = {key: info[key] for key in talents}
scores = sort_talents({'炮巡': 1, '驱逐': 2, '战列': 3}, talents)
print(sum([item['score'] for item in scores]), scores)

scores = sort_talents({'炮巡': 1, '轻雷巡': 2, '战列': 3}, talents)
print(sum([item['score'] for item in scores]), scores)

48.0 [{'score': 46.0, 'name': '既定的命运', 'scoreInfo': [('炮巡', 1, 8.0), ('驱逐', 2, 7.0), ('战列', 3, 8.0)]}, {'score': 2.0, 'name': '操舵手·小型舰', 'scoreInfo': [('驱逐', 2, 1.0)]}]
48.0 [{'score': 48.0, 'name': '既定的命运', 'scoreInfo': [('炮巡', 1, 8.0), ('轻雷巡', 2, 8.0), ('战列', 3, 8.0)]}]


In [18]:
show_talents({'正航': 2, '轻航': 1}, limit=15)

{'正航': 2, '轻航': 1}
 1: 30, 侵略如火  [('正航', 2, 10.0), ('轻航', 1, 10.0)]
 2: 21, 航空新兵·空母  [('正航', 2, 7.0), ('轻航', 1, 7.0)]
 3: 18, 其徐如林  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 4: 18, 不动如山  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 5: 18, 既定的命运  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 6: 18, 新晋指挥官·白鹰  [('正航', 2, 6.0), ('轻航', 1, 6.0)]
 7: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]
 8: 14, 王牌机师  [('正航', 2, 7.0)]
 9: 12, 新手整备士  [('正航', 2, 4.0), ('轻航', 1, 4.0)]
10: 12, 新晋指挥官·重樱  [('正航', 2, 4.0), ('轻航', 1, 4.0)]
11:  9, 其疾如风  [('正航', 2, 3.0), ('轻航', 1, 3.0)]
12:  9, 新手观测士·主力  [('正航', 2, 3.0), ('轻航', 1, 3.0)]
13:  6, 小小的奇迹  [('正航', 2, 2.0), ('轻航', 1, 2.0)]
14:  6, 对空炮手·主力  [('正航', 2, 2.0), ('轻航', 1, 2.0)]
15:  6, 轮机手·空母  [('正航', 2, 2.0), ('轻航', 1, 2.0)]



In [37]:
def cat_score(fleet, cat):
    member_count = sum(fleet.values())
    talents = {key: info[key] for key in get_talents(cat)}
    scores = sort_talents(fleet, talents)
    total_score = sum([item['score'] for item in scores]) / member_count
    print('{:5.3f}: {} for {}'.format(total_score, cat, fleet, ))
#     show_scores(scores)

In [20]:
get_talents('航 航樱')

['航空新兵·空母', '新晋指挥官·重樱']

In [21]:
get_talents('战 炮')

['炮击新手·主力']

In [22]:
get_talents('战 指空山')

['新晋指挥官·战列', '对空炮手·主力', '不动如山']

In [23]:
get_talents('驱 指耐装')

['新晋指挥官·驱逐', '轮机手·驱逐', '装填新手·驱逐']

In [24]:
get_talents('驱 运机')

['既定的命运', '操舵手·小型舰']

In [25]:
get_talents('驱 运机')

['既定的命运', '操舵手·小型舰']

In [40]:
cat_score({'驱逐': 3, '战列': 3}, '驱 运机')

8.000: 驱 运机 for {'驱逐': 3, '战列': 3}


In [41]:
cats = [
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
]
for cat in cats:
    cat_score({'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}, cat)

7.800: 战 指空山 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
1.600: 战 炮 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
4.800: 战 炮皇 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
2.200: 战 指耐空 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
2.800: 战 炮装 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}


In [427]:
cats = [
    '巡 雷炮',
    '巡 雷指',
    '巡 炮皇',
]
for cat in cats:
    cat_score({'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}, cat)

 24: 巡 雷炮 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 12, 新人雷击士·巡洋  [('轻雷巡', 2, 6.0)]
 2: 12, 炮击新手·巡洋  [('轻雷巡', 2, 6.0)]

 26: 巡 雷指 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 14, 新晋指挥官·巡洋  [('轻雷巡', 2, 7.0)]
 2: 12, 新人雷击士·巡洋  [('轻雷巡', 2, 6.0)]

 28: 巡 炮皇 for {'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}
 1: 16, 新晋指挥官·皇家  [('驱逐', 1, 3.0), ('轻雷巡', 2, 4.0), ('轻航', 1, 1.0), ('战列', 1, 4.0)]
 2: 12, 炮击新手·巡洋  [('轻雷巡', 2, 6.0)]



In [430]:
cats = [
    '驱 指耐装',
    '驱 雷机指',
    '驱 耐装炮中',
    '驱 指炮中',
]
for cat in cats:
    cat_score({'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}, cat)

 40: 驱 指耐装 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 14, 轮机手·驱逐  [('驱逐', 2, 7.0)]
 3: 10, 装填新手·驱逐  [('驱逐', 2, 5.0)]

 32: 驱 雷机指 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 14, 新人雷击士·驱逐  [('驱逐', 2, 7.0)]
 3:  2, 操舵手·小型舰  [('驱逐', 2, 1.0)]

 38: 驱 耐装炮中 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 14, 轮机手·驱逐  [('驱逐', 2, 7.0)]
 2: 10, 装填新手·驱逐  [('驱逐', 2, 5.0)]
 3: 10, 炮击新手·驱逐  [('驱逐', 2, 5.0)]
 4:  4, 新手观测士·先锋  [('驱逐', 2, 1.0), ('炮巡', 1, 2.0)]

 30: 驱 指炮中 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·驱逐  [('驱逐', 2, 8.0)]
 2: 10, 炮击新手·驱逐  [('驱逐', 2, 5.0)]
 3:  4, 新手观测士·先锋  [('驱逐', 2, 1.0), ('炮巡', 1, 2.0)]



In [431]:
cats = [
    '航 航樱',
    '航 樱指潜',
    '航 耐中指',
    '航 装机航',
]
for cat in cats:
    cat_score({'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}, cat)

 35: 航 航樱 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 21, 新晋指挥官·重樱  [('驱逐', 2, 4.0), ('炮巡', 1, 3.0), ('正航', 2, 4.0), ('战列', 1, 2.0)]
 2: 14, 航空新兵·空母  [('正航', 2, 7.0)]

 37: 航 樱指潜 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 21, 新晋指挥官·重樱  [('驱逐', 2, 4.0), ('炮巡', 1, 3.0), ('正航', 2, 4.0), ('战列', 1, 2.0)]
 2: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]

 32: 航 耐中指 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 16, 新晋指挥官·空母  [('正航', 2, 8.0)]
 2: 12, 新手观测士·主力  [('正航', 2, 3.0), ('战列', 1, 6.0)]
 3:  4, 轮机手·空母  [('正航', 2, 2.0)]

 24: 航 装机航 for {'驱逐': 2, '炮巡': 1, '正航': 2, '战列': 1}
 1: 14, 航空新兵·空母  [('正航', 2, 7.0)]
 2:  8, 新手整备士  [('正航', 2, 4.0)]
 3:  2, 操舵手·中型舰  [('炮巡', 1, 2.0)]



In [ ]:
cats = [
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
]
for cat in cats:
    cat_score({'驱逐': 1, '轻雷巡': 2, '轻航': 1, '战列': 1}, cat)

In [446]:
cats = [
    '驱 耐指',
    
    '驱 指耐装',
    '驱 雷机指',
    '驱 耐装炮中',
    '驱 指炮中',
    
    '战 指空山',
    '战 炮',
    '战 炮皇',
    '战 指耐空',
    '战 炮装',
    
    
    '航 航樱',
    '航 航装',
    '航 樱指潜',
    '航 耐中指',
    '航 装机航',
]
for cat in cats:
    cat_score({'驱逐': 3, '轻航': 1, '战列': 1}, cat)

 45: 驱 耐指 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]

 60: 驱 指耐装 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 3: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]

 48: 驱 雷机指 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 21, 新人雷击士·驱逐  [('驱逐', 3, 7.0)]
 3:  3, 操舵手·小型舰  [('驱逐', 3, 1.0)]

 54: 驱 耐装炮中 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 21, 轮机手·驱逐  [('驱逐', 3, 7.0)]
 2: 15, 装填新手·驱逐  [('驱逐', 3, 5.0)]
 3: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 4:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 42: 驱 指炮中 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 24, 新晋指挥官·驱逐  [('驱逐', 3, 8.0)]
 2: 15, 炮击新手·驱逐  [('驱逐', 3, 5.0)]
 3:  3, 新手观测士·先锋  [('驱逐', 3, 1.0)]

 39: 战 指空山 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1: 30, 不动如山  [('驱逐', 3, 6.0), ('轻航', 1, 6.0), ('战列', 1, 6.0)]
 2:  5, 新晋指挥官·战列  [('战列', 1, 5.0)]
 3:  4, 对空炮手·主力  [('轻航', 1, 2.0), ('战列', 1, 2.0)]

  8: 战 炮 for {'驱逐': 3, '轻航': 1, '战列': 1}
 1:  8, 炮击新手·主力  [('战列', 1, 8.0)]